### Detectar formacion de canal Bajista - Alcista

In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import yfinance as yf
import talib as ta
import matplotlib.pyplot as plt
import math
from datetime import date, timedelta
from scipy import stats
from scipy.signal import argrelextrema
from bokeh.plotting import figure, show, column
from bokeh.models import NumeralTickFormatter, Span
from sklearn.linear_model import LinearRegression

In [3]:
df_h = pd.read_csv(r"D:\traderxpro\data\dataxh.txt", sep="\t")
tickers = [
    'SPY',
    'META'
]

#    'SPY',
#    'META',
#    'AAPL',
#    'AMZN',
#    'NFLX',
#    'MRNA',
#    'TSLA',
#    'TNA',
#    'GLD',
#    'SLV',
#    'USO',
#    'BAC',
#    'CVX',
#    'XOM',
#    'QQQ'

In [4]:
df_h['datetime'] = pd.to_datetime(df_h['datetime'])
df = pd.DataFrame()
for ticker in tickers:
    company = df_h.query("companyName==@ticker").copy()
    company.sort_values(by=['datetime'])
    #company = yf.download(ticker, start = ini2_str, end = today_str, interval='60m')
    #company.columns = [company.columns[0][0], company.columns[1][0], company.columns[2][0], company.columns[3][0], company.columns[4][0]]
    #company.rename(columns={'Datetime':'Gmt time'}, inplace = True)
    #company['datetime'] = pd.to_datetime (company.index)
    #company['companyName'] = ticker
    company['SMA35'] = company['Close'].rolling(35).mean()
    company.dropna(inplace=False)
    company['SMA50'] = company['Close'].rolling(50).mean()
    company.dropna(inplace=False)
    company['EMA35'] = company['Close'].ewm(span=35, adjust=False).mean()
    company.dropna(inplace=False)
    company['EMA50'] = company['Close'].ewm(span=50, adjust=False).mean()
    company.dropna(inplace=False)
    
    #company['SMA100'] = company['Close'].rolling(100).mean()
    #company.dropna(inplace=False)
    #company['SMA200'] = company['Close'].rolling(200).mean()
    #company.dropna(inplace=False)
    
    df = pd.concat([df, company],ignore_index=True)

### Detect Pivots/Fractals

In [6]:
ord=15
ord2=5
#for ticker in tickers:
#print("====", ticker)
#dfpl = df[(df.companyName==ticker)] #df.query("companyName=='AAPL'")
max_idx = argrelextrema(df['Close'].values, np.greater, order=ord)[0]
min_idx = argrelextrema(df['Close'].values, np.less, order=ord)[0]

max_idx2 = argrelextrema(df['Close'].values, np.greater, order=ord2)[0]
min_idx2 = argrelextrema(df['Close'].values, np.less, order=ord2)[0]
#print(max_idx)
#print(min_idx)
#df['pivotHigh'] = np.nan
#df['pivotLow'] = np.nan
#df['isPivot'] = np.nan
#df['isPivot'] = np.nan
# Aplicar el cálculo solo a los índices en la lista
df.loc[max_idx, 'pivotHigh'] = df['High']+1e-3
df.loc[min_idx, 'pivotLow'] = df['Low']-(1e-3)
df.loc[max_idx, 'isPivot'] = 1
df.loc[min_idx, 'isPivot'] = 2

df.loc[max_idx2, 'pivotHigh2'] = df['High']+1e-3
df.loc[min_idx2, 'pivotLow2'] = df['Low']-(1e-3)
df.loc[max_idx2, 'isPivot2'] = 1
df.loc[min_idx2, 'isPivot2'] = 2

###### Canales bajistas, 
###### - Cuando el PM de 40 esta por encima del PM de 20 en direccion desendente
###### - Cuando se dan el cruce de medias moviles 35 y 50

In [8]:
#df = df.drop(columns=['cruce_medias']) # Drops column 'B'

In [9]:
# Señal: 1 = compra, -1 = venta, 0 = nada
#df['prev_SMA35'] = df['SMA35'].shift(1)
#df['prev_SMA50'] = df['SMA50'].shift(1)
df['prev_EMA35'] = df['EMA35'].shift(1)
df['prev_EMA50'] = df['EMA50'].shift(1)


#Cruce de medias
#df['cruce_medias'] = 0
#df.loc[(df['prev_SMA35'] < df['prev_SMA50']) & (df['SMA35'] > df['SMA50']), 'cruce_medias'] = 1  # Golden Cross (Compra)
#df.loc[(df['prev_SMA35'] > df['prev_SMA50']) & (df['SMA35'] < df['SMA50']), 'cruce_medias'] = -1 # Death Cross (Venta)

df['cruce_medias'] = 0
df.loc[(df['prev_EMA35'] < df['prev_EMA50']) & (df['EMA35'] > df['EMA50']), 'cruce_medias'] = 1  # Golden Cross (Compra)
df.loc[(df['prev_EMA35'] > df['prev_EMA50']) & (df['EMA35'] < df['EMA50']), 'cruce_medias'] = -1 # Death Cross (Venta)

In [10]:
df["cdlengulfing"] = ta.CDLENGULFING(df['Open'], df['High'], df['Low'], df['Close'])
df["cdlhammer"] = ta.CDLHAMMER(df['Open'], df['High'], df['Low'], df['Close'])
df["cdlmorningstar"] = ta.CDLMORNINGSTAR(df['Open'], df['High'], df['Low'], df['Close'])
df["cdlpiercing"] = ta.CDLPIERCING(df['Open'], df['High'], df['Low'], df['Close'])
df["cdlclosingmarubozu"] = ta.CDLCLOSINGMARUBOZU(df['Open'], df['High'], df['Low'], df['Close'])
df["cdlmarubozu"] = ta.CDLMARUBOZU(df['Open'], df['High'], df['Low'], df['Close'])
df["cdl3whitesoldiers"] = ta.CDL3WHITESOLDIERS(df['Open'], df['High'], df['Low'], df['Close'])
df["cdlharami"] = ta.CDLHARAMI(df['Open'], df['High'], df['Low'], df['Close'])
df["cdlharamicross"] = ta.CDLHARAMICROSS(df['Open'], df['High'], df['Low'], df['Close'])
df["cdlinvertdhammer"] = ta.CDLINVERTEDHAMMER(df['Open'], df['High'], df['Low'], df['Close'])
df["cdlladderbottom"] = ta.CDLLADDERBOTTOM(df['Open'], df['High'], df['Low'], df['Close'])

In [11]:
#df = df.drop(columns=['indcaso']) # Drops column 'B'

In [12]:
df_casos = pd.DataFrame()
caso = 0
ticker2 = ""
window=5
backeval=75
backevalTrend=0
dfpl = pd.DataFrame
trendH = []
trendL = []

ispivot=0
for i, row in df.iterrows():
    #if (i==512):
    cant = 0
    cant2 = 0
    ind_sl=0
    ind_trendHL=0

    ticker = df['companyName'][i]
    #Reinicio de casos por company
    if ticker2 != ticker:
        ticker2 = ticker
        caso = 0
    dfpl = df[(df.companyName==ticker)].loc[i-backeval:i+backeval].copy()
            
    #if (dfpl["cruce_medias"][i]==-1):
    #    #print("posible caso a la BAJA:",i, df['companyName'][i])
    #    ispivot=1
    #    ispivot2=2
    #    if (sl<0):
    #        #print("h1")
    #        ind_sl=1
    #
    #elif (dfpl["cruce_medias"][i]==1):
    #    #print("posible caso a la ALZA:",i, df['companyName'][i])
    #    ispivot=2
    #    ispivot2=1
    #    if (sl>0):
    #        #print("h2")
    #        ind_sl=1
    ind_sl=0
    tipo=""
    if ((dfpl["cruce_medias"][i]==-1) | (dfpl["cruce_medias"][i]==1)):
        if (dfpl["cruce_medias"][i]==-1):
           tipo="BAJA" 
        else:
            tipo="ALZA"
        print("posible caso:",i, df['companyName'][i], "==>", tipo)

        #Se dio el cruce de medias, antes existio un pivot high aprox 2 dias como maximo
        fecha= dfpl['datetime'][i]
        #fecha_limite = dfpl['datetime'][i] - pd.Timedelta(days=2) #resta dias calendarios
        fecha_limite = dfpl['datetime'][i] - pd.offsets.BDay(2) #resta dias laborables
        #print("fecha:", dfpl['datetime'][i] ,", fecha_limite:", fecha_limite)
        
        #cnt1 = dfpl.query("isPivot==@ispivot and datetime>=@fecha_limite and datetime<=@fecha")["isPivot"].count()
        #cnt2 = dfpl.query("isPivot==@ispivot2 and datetime>=@fecha_limite and datetime<=@fecha")["isPivot"].count()

        cnt1=1
        cnt2=0

        indiceFinal=0
        
        #print("cantidad:", cnt)
        if (cnt1>0 and cnt2==0):
            #Obtener casos      
            #print("h00")
            if (dfpl["cruce_medias"][i]==1): #ALCISTA               
                #ALZA, velas por encima de promedios moviles
                #ultimo high por encima y ultimo low cerca a los promedios
                siguiente_H = dfpl[(dfpl.index>=i) & ((dfpl["isPivot"]==1) | (dfpl["isPivot2"]==1))].head(1)
                if (siguiente_H.shape[0]>0):
                    if (siguiente_H.iloc[0]['High']>siguiente_H.iloc[0]['EMA35']):
                        indice = siguiente_H.index[0]
                        siguiente_L = dfpl[(dfpl.index>indice) & ((dfpl["isPivot"]==2) | (dfpl["isPivot2"]==2))].head(1)
                        if (siguiente_L.shape[0]>0):
                            if (((siguiente_L.iloc[0]['Low']-siguiente_L.iloc[0]['EMA35'])<1) &  (siguiente_L.iloc[0]['Low']<siguiente_H.iloc[0]['High'])):
                                indiceFinal = siguiente_L.index[0]

            elif (dfpl["cruce_medias"][i]==-1): #BAJISTA                    
                #BAJA, velas por debajo de promedios moviles
                #ultimo low por debajo y ultimo high cerca a los promedios
                #print("h01")
                siguiente_L = dfpl[(dfpl.index>=i) & ((dfpl["isPivot"]==2) | (dfpl["isPivot2"]==2)) ].head(1)
                if (siguiente_L.shape[0]>0):
                    #print("h02")
                    if (siguiente_L.iloc[0]['Low']<siguiente_L.iloc[0]['EMA35']):
                        #print("h03")
                        indice = siguiente_L.index[0]
                        #print("indice:", indice)
                        siguiente_H = dfpl[(dfpl.index>indice) & ((dfpl["isPivot"]==1) | (dfpl["isPivot2"]==1))].head(1)
                        if (siguiente_H.shape[0]>0):
                            #print("h04:",siguiente_H.index[0])
                            if (((siguiente_H.iloc[0]['EMA35']-siguiente_H.iloc[0]['High'])<1) & (siguiente_H.iloc[0]['High']>siguiente_L.iloc[0]['Low'])):
                                #print("h05")
                                indiceFinal = siguiente_H.index[0]

            #print ("indiceFinal:", indiceFinal)
            if (indiceFinal>0):
                #print("h0")
                ultimos_2H = dfpl[(dfpl["isPivot"]==1) | (dfpl["isPivot2"]==1)].loc[:indiceFinal].tail(2)
                ultimos_2L = dfpl[(dfpl["isPivot"]==2) | (dfpl["isPivot2"]==2)].loc[:indiceFinal].tail(2)
                    
                if (ultimos_2H.shape[0]==2 & ultimos_2L.shape[0]==2):
                    #print("h1")
                    penultimo_valorH = ultimos_2H.iloc[0]['High']
                    ultimo_valorH = ultimos_2H.iloc[1]['High']
                    penultimo_valorL = ultimos_2L.iloc[0]['Low']
                    ultimo_valorL = ultimos_2L.iloc[1]['Low']

                    #En caso de tendencia ALCISTA
                    #tomar los 2 ultimos HH – Higher High (Maximos mas Altos)
                    #tomar los 2 ultimos HL – Higher Low (Maximos mas bajos)
                    if (dfpl["cruce_medias"][i]==1): #ALCISTA
                        #print("h2")
                        if ((ultimo_valorH>penultimo_valorH) | (ultimo_valorL>penultimo_valorL)):
                            #print("h21")
                            ind_trendHL=1

                    #En caso de tendencia BAJISTA
                    #tomar los 2 ultimos LH – Lower High (Minimos mas Altos)
                    #tomar los 2 ultimos LL – Lower Low (Minimos mas bajos)
                    elif (dfpl["cruce_medias"][i]==-1): #BAJISTA
                        #print("h3")
                        if ((ultimo_valorH<penultimo_valorH) | (ultimo_valorL<penultimo_valorL)):
                            #print("h31")
                            ind_trendHL=1

            #print("indiceFinal:",indiceFinal,",ind_trendHL:",ind_trendHL)

            if (indiceFinal>0):
                #Inicio determinar slope que indica tendencia al ALZA o a la BAJA
                medias35 = dfpl.loc[i:indiceFinal].EMA35.values
                idxmedias35 = dfpl.loc[i:indiceFinal].EMA35.index
                medias50 = dfpl.loc[i:indiceFinal].EMA50.values
                idxmedias50 = dfpl.loc[i:indiceFinal].EMA50.index
                if ((len(medias35)>=2) and  (len(medias50)>=2)):
                    sl35, interc35, r_value35, _, _ = stats.linregress(idxmedias35,medias35)
                    sl50, interc50, r_value50, _, _ = stats.linregress(idxmedias50,medias50)
                #Fin slope
        
                if (dfpl["cruce_medias"][i]==1): #ALCISTA
                    #Revision de pendiente
                    if (sl35>0 and sl50>0 #Pendiente positiva
                       #and np.isclose(sl35, sl50, atol=1e-5) #Verificar si son paralelas
                       #and interc35>interc50 #La línea 35 está por encima de la línea 50
                       ):
                        ind_sl=1
                elif (dfpl["cruce_medias"][i]==-1): #BAJISTA 
                    #Revision de pendiente
                    if (sl35<0 and sl50<0 #Pendiente negativa
                        #and np.isclose(sl35, sl50, atol=1e-5) #Verificar si son paralelas
                        #and interc50>interc35 #La línea 50 está por encima de la línea 35
                       ):
                        ind_sl=1
    
            #print("sl35:",sl35,"sl50:",sl50)
            #print("interc35:",interc35,"interc50:",interc50, ", ind_sl:", ind_sl)

            
            
            if ((indiceFinal>0) and (ind_trendHL>0) and (ind_sl>0)):
                #print("h4")
                #INSERT CASO
                caso=caso+1
                dfpl.loc[i,'ind_posicion']=0
               

                #Final de caso es siguiente Pivot
                if (dfpl["cruce_medias"][i]==1): #ALCISTA
                    dfpl.loc[indiceFinal,'isBreakOutIni']=1
                    k=0
                    while (k<=3):
                        cnt2 = dfpl.query("index>@indiceFinal and isPivot==1").shape[0]
                        if cnt2>0:        
                            id2 = dfpl.query("index>@indiceFinal and isPivot==1").index[0]
                            dfpl.loc[id2,'isBreakOutFinal'] = 1
                            k=4
                        else:
                            idfinal = dfpl.index[-1] 
                            idfinal2 = idfinal+25
                            if idfinal2 in df[df['companyName']==ticker].index:
                                dfpl2 = df[(df.companyName==ticker)].loc[idfinal+1:idfinal2].copy()
                                dfpl = pd.concat([dfpl, dfpl2],ignore_index=False)                           
                            else:                  
                                dfpl['isBreakOutFinal'] = np.nan
                            k=k+1
                           
                
                        
                elif (dfpl["cruce_medias"][i]==-1): #BAJISTA
                    dfpl.loc[indiceFinal,'isBreakOutIni']=-1
                    k=0
                    while (k<=3):
                        cnt2 = dfpl.query("index>@indiceFinal and isPivot==2").shape[0]
                        if cnt2>0:        
                            id2 = dfpl.query("index>@indiceFinal and isPivot==2").index[0]
                            dfpl.loc[id2,'isBreakOutFinal'] = -1
                            k=4
                        else:
                            idfinal = dfpl.index[-1] 
                            idfinal2 = idfinal+25
                            if idfinal2 in df[df['companyName']==ticker].index:
                                dfpl2 = df[(df.companyName==ticker)].loc[idfinal+1:idfinal2].copy()
                                dfpl = pd.concat([dfpl, dfpl2],ignore_index=False)                           
                            else:                  
                                dfpl['isBreakOutFinal'] = np.nan
    
                            k=k+1

                
                if (df["cruce_medias"][i]==-1):                    
                    df.loc[i,'ind_posicion']=-1                    
                    df.loc[indiceFinal,'isBreakOutIni']=-1
                    df.loc[id2,'isBreakOutFinal']=-1
                elif (df["cruce_medias"][i]==1):                        
                    df.loc[i,'ind_posicion']=1                   
                    df.loc[indiceFinal,'isBreakOutIni']=1
                    df.loc[id2,'isBreakOutFinal']=1

                #print("h5")
                dfpl["caso"] = caso
                print(i, df['companyName'][i], ", cruce medias:",df["cruce_medias"][i],", slope35:", sl35, ", slope50:", sl50)
                if len(df_casos)<1:
                    df_casos = dfpl
                else:
                    df_casos = pd.concat([df_casos, dfpl],ignore_index=False)

posible caso: 116 SPY ==> BAJA
116 SPY , cruce medias: -1 , slope35: -0.01664558094338607 , slope50: -0.016549370147725048
posible caso: 127 SPY ==> ALZA
127 SPY , cruce medias: 1 , slope35: 0.21773070179993398 , slope50: 0.17567901098930683
posible caso: 296 SPY ==> BAJA
296 SPY , cruce medias: -1 , slope35: -0.18260069239277693 , slope50: -0.14115856762456722
posible caso: 424 SPY ==> ALZA
424 SPY , cruce medias: 1 , slope35: 0.17555931383133686 , slope50: 0.1670111630075893
posible caso: 478 SPY ==> BAJA
posible caso: 479 SPY ==> ALZA
479 SPY , cruce medias: 1 , slope35: 0.041496018790358025 , slope50: 0.03303738684991981
posible caso: 512 SPY ==> BAJA
512 SPY , cruce medias: -1 , slope35: -0.15445522331443726 , slope50: -0.12313649778838845
posible caso: 626 SPY ==> ALZA
posible caso: 680 SPY ==> BAJA
posible caso: 748 SPY ==> ALZA
posible caso: 1033 SPY ==> BAJA
1033 SPY , cruce medias: -1 , slope35: -0.11455394346089216 , slope50: -0.09734176410930492
posible caso: 1062 SPY ==> A

In [13]:
#df_casos[['companyName', 'caso']].unique()
#resultado = df_casos.groupby(['companyName', 'caso'])['Close'].count().reset_index()
#resultado[resultado['companyName']=="META"].shape[0]

In [14]:
#CODIGO DE REVISION CARLOS
#dfpl = df.iloc[500:600]
dfpl = df[df["companyName"]=="SPY"]

dfpl["Datetime_str"] = dfpl["datetime"].astype(str)

p = figure(width=2500, height=500,
        title="Canal Bajista - Alcista",
        background_fill_color="#efefef",
        tooltips=[("Index", "@index"),("datetime", "@Datetime_str"), ("Open", "@Open"), ("High","@High"), ("Low","@Low"), ("Close","@Close"),
                    ("EMA35", "@EMA35"),
                    ("EMA50", "@EMA50"),
                    ("SMA35", "@SMA35"),
                    ("SMA50", "@EMA50"),
                    ("cdlengulfing","@cdlengulfing"), 
                    ("cdlhammer","@cdlhammer"), 
                    ("cdlmorningstar","@cdlmorningstar"), 
                    ("cdlpiercing","@cdlpiercing"), 
                    ("cdlclosingmarubozu","@cdlclosingmarubozu"), 
                    ("cdlmarubozu","@cdlmarubozu"), 
                    ("cdl3whitesoldiers","@cdl3whitesoldiers"), 
                    ("cdlharami","@cdlharami"), 
                    ("cdlharamicross","@cdlharamicross"), 
                    ("cdlinvertdhammer","@cdlinvertdhammer"), 
                    ("cdlladderbottom","@cdlladderbottom")]
        #tooltips=[("Index", "@index"), ("Open", "@Open"), ("High","@High"), ("Low","@Low"), ("Close","@Close")]
        )
p.xaxis.major_label_orientation = 0.8 # radians
p.x_range.range_padding = 0.05
#p.xaxis.axis_line_join = "bevel" # radians
p.xaxis.axis_line_width = 2

p.segment("index", "High", "index","Low",  color="black", line_width=1, source=dfpl)

inc = dfpl.query("Close>Open")
dec = dfpl.query("Open>Close")

p.vbar(    
    x="index",
    width=0.6,
    bottom="Open",
    top="Close",
    fill_color="red",
    line_color="red",    
    source=dec   
)


p.vbar(    
    x="index",
    width=0.6,
    bottom="Open",
    top="Close",
    fill_color="green",
    line_color="green", 
    source=inc   
)

p.line(
    x="index", 
    y="SMA35", 
    color="white",
    legend_label="SMA35",
    source=dfpl)


p.line(
    x="index", 
    y="SMA50", 
    color="blue",
    legend_label="SMA50",
    source=dfpl)


p.line(
    x="index", 
    y="EMA35", 
    color="black",
    legend_label="EMA35",
    source=dfpl)


p.line(
    x="index", 
    y="EMA50", 
    color="red",
    legend_label="EMA50",
    source=dfpl)

#p.line(
#    x="index", 
#    y="SMA100", 
#    color="green",
#    legend_label="SMA100",
#    source=dfpl)
#
#p.line(
#    x="index", 
#    y="SMA200", 
#    color="purple",
#    legend_label="SMA200",
#    source=dfpl)



p.scatter(x="index", y="Low", marker="triangle", size=20,
           line_color="navy", fill_color="blue", alpha=0.5, legend_label="inicio1", source=dfpl[(dfpl.isBreakOutIni==1)])

p.scatter(x="index", y="High", marker="inverted_triangle", size=20,
           line_color="navy", fill_color="red", alpha=0.5, legend_label="inicio2", source=dfpl[(dfpl.isBreakOutIni==-1)])


p.scatter(x="index", y="High", marker="inverted_triangle", size=20,
           line_color="navy", fill_color="black", alpha=0.5, legend_label="fin1", source=dfpl[(dfpl.isBreakOutFinal==1)])

p.scatter(x="index", y="Low", marker="triangle", size=20,
           line_color="navy", fill_color="brown", alpha=0.5, legend_label="fin2", source=dfpl[(dfpl.isBreakOutFinal==-1)])




p.scatter(x="index", y="pivotHigh", marker="circle", size=5,
           line_color="navy", fill_color="blue", alpha=0.5, legend_label="Cambio Tendencia Bajista1", source=dfpl[(dfpl.isPivot==1)])

p.scatter(x="index", y="pivotLow", marker="circle", size=5,
           line_color="navy", fill_color="green", alpha=0.5, legend_label="Cambio Tendencia Alcista1", source=dfpl[(dfpl.isPivot==2)])


p.scatter(x="index", y="pivotHigh2", marker="diamond", size=8,
           line_color="navy", fill_color="blue", alpha=0.5, legend_label="Cambio Tendencia Bajista2", source=dfpl[(dfpl.isPivot2==1) & (dfpl.isPivot.isna()) ])

p.scatter(x="index", y="pivotLow2", marker="diamond", size=8,
           line_color="navy", fill_color="green", alpha=0.5, legend_label="Cambio Tendencia Alcista2", source=dfpl[(dfpl.isPivot2==2) & (dfpl.isPivot.isna())])

p.yaxis[0].formatter = NumeralTickFormatter(format="$0.00")
p.xaxis.axis_label = "Fecha"
p.yaxis.axis_label = "Precio"
p.legend.location="top_left"
p.legend.click_policy="hide"

#Render linea vertical
alza = (dfpl[(dfpl.ind_posicion==1)].index).tolist()
baja = (dfpl[(dfpl.ind_posicion==-1)].index).tolist()
#vline=Span(source=inicio,dimension='height', line_color='blue',line_width=0.8, line_dash_offset= 0, line_dash='dashed', name="hola esto es una prueba", level='annotation', tags= ['square'])
#p.renderers.extend([vline])

p.vspan(
    x=alza,
    line_color="blue"
)

p.vspan(
    x=baja,
    line_color="red"
)

show(p)

C:\Users\carlo\AppData\Local\Temp\ipykernel_5472\799839678.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfpl["Datetime_str"] = dfpl["datetime"].astype(str)


In [15]:
#descarga de informacion, estrategia: RUPTURA DEL CANAL BAJISTA
import os
#path = r'C:\Users\carlo\OneDrive\Documentos\TRADER\traderapp\data\rcb_h.txt'
path = r'D:\traderxpro\data\cba_h.txt'
# check whether the file exists
if os.path.exists(path):
    # delete the file
    os.remove(path)
else:
    # if the file does not exist.
    print("File does not exists. File needs to be created.")

#export DataFrame to text file
with open(path, 'a') as f:
    #df_string = appl_hor3.to_string(header=True, index=False, sep ='\t')
    df_casos.to_csv(path, header=True, index=None, sep='\t', mode='w')
    #f.write(df_string)

#### BACKTESTING
###### Usando paquete backtesting.py

In [25]:
from backtesting import Backtest, Strategy

C:\Users\carlo\anaconda3\Lib\site-packages\backtesting\_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [26]:
df_backTesting = df.copy()
df_backTesting.set_index('datetime', inplace=True)

In [27]:
#Strategia
class strategyCanalBajistaAlcista(Strategy):    
    def init(self):        
        self.breakout_entry = self.I(lambda x: x, self.data.isBreakOutIni) #Indica la señal entrada al backtesting
        self.breakout_exit = self.I(lambda x: x, self.data.isBreakOutFinal) #Indica la señal salida al backtesting

    def next(self):       
        #print(self.data)        
        if self.breakout_entry[-1] == 1:
            self.position.close()
            self.buy()  #COMPRA
        elif self.breakout_entry[-1] == -1:
            self.position.close()
            self.sell()  #VENTA
        elif self.breakout_exit[-1] == 1 and self.position:
            self.position.close() #CIERRE
        elif self.breakout_exit[-1] == -1 and self.position:
            self.position.close() #CIERRE

In [28]:
# Por cada TIcket
my_stats = {}
estadisticas = pd.DataFrame()          # lista para ticker
tradesprev = pd.DataFrame()     # lista para cada caso por ticker
for ticker in df_backTesting['companyName'].unique():
    ticker_data = df_backTesting[df_backTesting['companyName'] == ticker].copy() #Filtro por cada ticker
    #ticker_data=ticker_data.sort_values(by='caso')
    #print("\n" + "="*50 + "\n")
    bt = Backtest(ticker_data, strategyCanalBajistaAlcista, cash=10_000)
    #stats = bt.run()
    #my_stats[f"stats_BreakOUT_{ticker}"] = bt.run()
    my_stats = bt.run()
    listTrades = my_stats['_trades']
    tradesdf = pd.DataFrame(listTrades) #trades
    tradesdf['Ticker'] = ticker
    my_stast_filter=pd.DataFrame([my_stats])
    if tradesprev.shape[0]==0:
        tradesprev = tradesdf.copy()
        estadisticas=my_stast_filter.copy()
    else:
        tradesprev = pd.concat([tradesprev, tradesdf],ignore_index=True)
        estadisticas = pd.concat([estadisticas, my_stast_filter],ignore_index=True)
    # Mostramos
    print(f"Resultados para {ticker}:")
    print(stats)   
    #plt.title(f"Backtest -  {ticker}")
    bt.plot(resample=False, filename=f"plot_BreakOUT_{ticker}")

Backtest.run:   0%|          | 0/3406 [00:00<?, ?bar/s]

Resultados para SPY:
<module 'scipy.stats' from 'C:\\Users\\carlo\\anaconda3\\Lib\\site-packages\\scipy\\stats\\__init__.py'>


C:\Users\carlo\AppData\Local\Temp\ipykernel_5472\3604966662.py:27: UserWarning: Superimposed OHLC plot matches the original plot. Skipping.
  bt.plot(resample=False, filename=f"plot_BreakOUT_{ticker}")


Backtest.run:   0%|          | 0/3480 [00:00<?, ?bar/s]

Resultados para META:
<module 'scipy.stats' from 'C:\\Users\\carlo\\anaconda3\\Lib\\site-packages\\scipy\\stats\\__init__.py'>


C:\Users\carlo\AppData\Local\Temp\ipykernel_5472\3604966662.py:27: UserWarning: Superimposed OHLC plot matches the original plot. Skipping.
  bt.plot(resample=False, filename=f"plot_BreakOUT_{ticker}")
